In [ ]:
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [ ]:
yc_data = pd.read_csv('../data/yc_time_series.csv', parse_dates=True, index_col=0)
tenors = list(yc_data.columns)

In [ ]:
train_start, train_end = '2006-03-01', '2016-12-31'
test_start, test_end = '2017-01-01', '2018-07-30'

train_data = yc_data.loc[train_start:train_end].dropna(axis=0)
test_data = yc_data.loc[test_start:test_end].dropna(axis=0)

In [ ]:
X_train = train_data.values
X_test = test_data.values
X_train.shape, X_test.shape

In [ ]:
from sklearn.decomposition import PCA

k = 2
pca_model = PCA(n_components=k)
pca_coeffs = pca_model.fit_transform(X_train)

# predictions on test data
pca_preds = pca_model.inverse_transform(pca_model.transform(X_test))

In [ ]:
from keras.models import load_model
ae = load_model('../pretrained_models/yield_curve_autoencoder/autoencoder_model.h5')

In [ ]:
from ipywidgets import *
from bqplot import *
import bqplot.pyplot as plt
from bqplot.colorschemes import *
from bqplot.interacts import IndexSelector

learning_phase_map = {'Train': train_data, 'Test': test_data}

ts_fig = plt.figure(title='Time Series Of Treasury Yield Curve Members', 
                    layout=Layout(width='1400px', height='500px'))
plt.scales(scales={'x': DateScale()})
axes_options = {'x': {'label': 'Dates'}, 
                'y': {'tick_format': '.1%', 
                      'label_location': 'end', 
                      'label_offset': '-2ex'}}
time_series = plt.plot([], [], colors=CATEGORY20c, labels=tenors,
                       labels_visibility='label', apply_clip=False,
                       axes_options=axes_options)

# create a date selector using IndexSelector
date_selector = IndexSelector(scale=time_series.scales['x'], 
                              marks=[time_series])
ts_fig.interaction = date_selector

# yield curve fig
title_tmpl = 'Yield Curve Reconstruction For: {}'
yc_fig = plt.figure(title=title_tmpl.format(''), 
                    legend_location='top-left')
yc_fig.layout.width = '1000px'
yc_fig.layout.height = '600px'
plt.scales(scales={'y': LinearScale()})
axes_options = {'x': {'label': 'Tenor'},
                'y': {'label': 'Yield', 
                      'tick_format': '.1%',
                      'label_location': 'end', 
                      'label_offset': '-2ex'}}
yc_lines = plt.plot(tenors, [], display_legend=True, 
                    marker='square', marker_size=32, 
                    colors=['yellow', 'limegreen', 'magenta'],
                    axes_options=axes_options, labels=['True', 'PCA', 'AE'])

# train/test dropdown
learning_phase_dd = Select(options=['Train', 'Test'], description='Learning Phase')

# callbacks
def update_yield_curves(*args):
    global data, ae_yield_curves, pca_yield_curves, true_yield_curves
    if time_series.selected:
        sel_idx = time_series.selected[0]
        sel_date = data.index[sel_idx]

        true_yield_curve = true_yield_curves[sel_idx]
        pca_yield_curve = pca_yield_curves[sel_idx]
        ae_yield_curve = ae_yield_curves[sel_idx]

        yc_lines.y = np.row_stack([true_yield_curve, pca_yield_curve, ae_yield_curve])
        yc_fig.title = title_tmpl.format(sel_date.strftime('%Y-%m-%d'))

# date_selector updates the 'selected' attribute of the time_series mark
# on which we register the callback 'update_yield_curves'
time_series.observe(update_yield_curves, 'selected')

def update_time_series(*args):
    global data, ae_yield_curves, pca_yield_curves, true_yield_curves
    data = learning_phase_map[learning_phase_dd.value]
    
    with time_series.hold_sync():
        time_series.x = data.index
        time_series.y = data.values.T
    yc_lines.y = []
    
    X = data.values    
    ae_yield_curves = ae.predict(X)
    pca_yield_curves = pca_model.inverse_transform(pca_model.transform(X))
    true_yield_curves = X

learning_phase_dd.observe(update_time_series, 'value')
update_time_series(None)

VBox([learning_phase_dd, ts_fig, yc_fig])